In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold
import os
import os, shutil 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
data = pd.read_csv(path+'icml_face_data.csv')
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
classes = dict(zip(range(0, 7), (((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index())/len(data[data[' Usage']=='Training']['emotion'])).tolist()))

In [ ]:
X = data.emotion
kf = KFold(n_splits=6)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

In [ ]:
print(data)

In [ ]:
# Function to parse data into right format
# Output: Image in right shaped and normalized + labels
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48)) # 1
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48)) # 1
        image_array[i] = image
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])

In [ ]:
train_images = train_imgs.reshape((train_imgs.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_imgs.reshape((val_imgs.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_imgs.reshape((test_imgs.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255


In [ ]:
print("train shape", np.shape(train_imgs))
print("validation shape", np.shape(val_imgs))
print("validatio shape", np.shape(val_imgs))

In [ ]:
print(train_imgs)

MLP model

In [ ]:
# Building a MLP model based on LeNet architecture 
model_mlp = keras.Sequential()
model_mlp.add(layers.Flatten(input_shape=(48, 48, 1)))
model_mlp.add(layers.Dense(units=120, activation='relu'))
model_mlp.add(layers.Dense(units=84, activation='relu'))
model_mlp.add(layers.Dense(units=7, activation = 'softmax'))
model_mlp.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])
model_mlp.summary()

In [ ]:
# Training the model, and validating
model_mlp.fit(train_imgs, train_lbls, 
          epochs=10, batch_size=32, 
          validation_data=(val_imgs, val_lbls), verbose=1)

In [ ]:
from keras.utils import to_categorical
train_labels = to_categorical(train_lbls)
val_labels = to_categorical(val_lbls)
test_labels = to_categorical(test_lbls)

CNN model

In [ ]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='selu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='selu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu'))
model_cnn.add(layers.Dense(7, activation='sigmoid'))

In [ ]:
model_cnn.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()

In [ ]:
history = model_cnn.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = classes,
                    epochs=12,
                    batch_size=512)

In [ ]:
#Train accuracy and validation accuracy vs epoch graph
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_prob = model_cnn.predict(test_images)
test_pred = np.argmax(test_prob, axis=1)
test_accuracy = np.mean(test_pred == test_lbls)

print(test_accuracy)

In [ ]:
conf_mat = confusion_matrix(test_lbls, test_pred)
pd.DataFrame(conf_mat, columns=emotions.values(), index=emotions.values())

1. У чому призначення різних типів шарів згорткових нейромереж?<br>
Згортковий шар є основним будівельним блоком СНС.
Агрегуючий шар служить для поступового скорочення просторового розміру уявлення для зменшення кількості параметрів і обсягу обчислень в мережі, а тому ще й для контролю перенавчання.
Шар зрізаних лінійних вузлів підсилює нелінійні властивості функції прийняття рішення і мережі в цілому, зачіпаючи рецептивні поля сверточного шару.
Нейрони в  шарі мають сполучення з усіма возбуждениями попереднього шару, як це можна бачити в звичайних нейронних мережах. Їх порушення потім може бути обчислено матричних множенням, за яким слід зміщення упередженості.
Шар втрат визначає, як тренування штрафує відхилення між передбачуваними і справжніми знаками, і є, як правило, завершальним шаром. Для різних завдань в ньому можуть використовувати різні функції втрат (найчастіше це softmax і sigmoid).
2. Які основні етапи типового проекту машинного навчання?<br>
Бізнес-аналіз, аналіз і підготовка даних, моделювання, оцінка рішення, впровадження.